In [93]:
import pandas as pd
import numpy as np
import regex as re

In [52]:
df = pd.read_csv('dataset/results.csv')

In [101]:
df

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False
...,...,...,...,...,...,...,...,...,...
44552,2023-03-28,Haiti,Bermuda,3,1,CONCACAF Nations League,San Cristóbal,Dominican Republic,True
44553,2023-03-28,Guyana,Montserrat,0,0,CONCACAF Nations League,Bridgetown,Barbados,True
44554,2023-03-28,Bonaire,Turks and Caicos Islands,1,2,CONCACAF Nations League,Rincon,Bonaire,False
44555,2023-03-28,United States Virgin Islands,Sint Maarten,1,2,CONCACAF Nations League,Saint Croix,United States Virgin Islands,False


In [87]:
wc_data = df[df.tournament == 'FIFA World Cup'].copy()

def get_result(row,side):
    if side == 'home':
        if row.home_score > row.away_score:
            return 3
        elif row.home_score < row.away_score:
            return 0
        else:
            return 1
    else:
        if row.home_score < row.away_score:
            return 3
        elif row.home_score > row.away_score:
            return 0
        else:
            return 1
        
def get_goal_diff(row,side):
    if side == 'home':
        return row.home_score - row.away_score
    else:
        return row.away_score - row.home_score
    
def return_world_cup_result(data,country_name):
    df = data.copy()
    df = df[((df.tournament == 'FIFA World Cup') & (df.country == country_name))].copy()

    df['home_result'] = df.apply(lambda x: get_result(x,'home'),axis=1)
    df['away_result'] = df.apply(lambda x: get_result(x,'away'),axis=1)
    df['home_goal_diff'] = df.apply(lambda x: get_goal_diff(x,'home'),axis=1)
    df['away_goal_diff'] = df.apply(lambda x: get_goal_diff(x,'away'),axis=1)

    home = df[['tournament','country','home_team','home_result','home_goal_diff']].copy()
    away = df[['tournament','country','away_team','away_result','away_goal_diff']].copy()

    home.rename(columns = {'home_team':'team','home_result':'result','home_goal_diff':'goal_diff'}, inplace=True)
    away.rename(columns = {'away_team':'team','away_result':'result','away_goal_diff':'goal_diff'}, inplace=True)
    
    combined = pd.concat([home,away]).copy()

    # Match count is the best indicator of how well the team did
    combined['match_count'] = 1
    
    # This ordering is not the most accurate, but is systematic way to calculate power rating
    combined = combined.groupby(['tournament','country','team'],as_index=False
                    ).sum().sort_values(by=['match_count','result','goal_diff'],ascending=False
                                       ).reset_index(drop=True).reset_index().rename(columns={'index':'rank'})
    combined['wc_result_score']  = ( 32 - ( combined['rank'] + 1 ) ) / 32
    
    return combined


In [100]:
russia = return_world_cup_result(wc_data,'Russia')[['country','team','wc_result_score']]
qatar = return_world_cup_result(wc_data,'Qatar')[['country','team','wc_result_score']]

wc_results_2018_2022 = russia.merge(qatar, how='outer', on=['team']).copy()

wc_results_2018_2022 = wc_results_2018_2022[['team','wc_result_score_x','wc_result_score_y']]

wc_results_2018_2022 = wc_results_2018_2022.fillna(0)

wc_results_2018_2022 = wc_results_2018_2022.melt(id_vars=["team"])

wc_results_2018_2022 = wc_results_2018_2022.drop(columns=['variable']).rename(columns={'value':'power_rating'})

wc_results_2018_2022 = wc_results_2018_2022.groupby(['team'],as_index=False).mean()

wc_results_2018_2022 = wc_results_2018_2022.rename(columns={'team':'country_name'})

,team,power_rating
0,Argentina,0.718750
1,Australia,0.406250
2,Belgium,0.625000
3,Brazil,0.796875
4,Cameroon,0.203125
5,Canada,0.015625
6,Colombia,0.359375
7,Costa Rica,0.125000
8,Croatia,0.890625
9,Denmark,0.406250


In [102]:
wc_results_2018_2022.to_csv('world_cup_result.csv')